In [6]:
import sys
import os
import time
import numpy as np
from snAPI.Main import *
import Motor_f as mot

In [7]:
def XY_scan(handlex,handley,x0,xf,Nx,y0,yf,Ny,wait,folder):# positions in micrometers
    
    #Create the scanning pattern (it is important to be ordered to minimize displacements)
    xs=np.linspace(x0,xf,Nx)
    ys=np.linspace(y0,yf,Ny)
    XY=np.meshgrid(xs,ys)
    Order=np.zeros_like(XY[0])
    for i in range(0,np.shape(XY[0])[0]):
        for j in range(0,np.shape(XY[0])[1]):
            if j%2==0:
                Order[i,j]=i+Ny*j
            else:
                Order[i,j]=Nx-i-1+Ny*j
    xx,yy,o=XY[0].flatten(),XY[1].flatten(),Order.flatten()
    xsorted=[x for _, x in sorted(zip(o,xx))]
    ysorted=[x for _, x in sorted(zip(o, yy))]
    pairs=np.vstack((xsorted,ysorted)).T
    # The points of the scan are saved at pairs (ordered)
    i=0
    for point in pairs:
        
        mot.move_abs_xy(handlex,point[0]/1000) #move functions work in milimeters
        mot.move_abs_xy(handley,point[1]/1000)
        time.sleep(wait)
        sn.setPTUFilePath("./Data/{}/PTU_{:.2f}_{:.2f}.ptu".format(folder,point[0],point[1]))
        sn.histogram.setRefChannel(0)
        sn.histogram.setBinWidth(1000)
        sn.histogram.measure(1000,savePTU=True)
        sn.histogram.getData()
        
        i+=1
                
    return True

In [8]:
xdrive=mot.connectX()
ydrive=mot.connectY()
zdrive=mot.connectZ()

---- Connecting PDXC motor ----
Devices=[['01403418', '0403&6015&THORLABS PDXC 1.51&THORLABS&COM11&COM'], ['01404934', '0403&6015&THORLABS PDXC 1.51&THORLABS&COM10&COM']]
connect  01403418
open  01403418  failed
---- Connecting PDXC motor ----
Devices=[['01403418', '0403&6015&THORLABS PDXC 1.51&THORLABS&COM11&COM'], ['01404934', '0403&6015&THORLABS PDXC 1.51&THORLABS&COM10&COM']]
connect  01404934
open  01404934  failed


SerialPortBusyException: SerialPortBusyException: Cannot open serial port: Port is likely already opened by another application. Please close the application that is using the port.

In [9]:
sn = snAPI(libType=LibType.HH)
sn.closeDevice()

sn.getDevice()
sn.initDevice(MeasMode.T2)

True

In [12]:
sn.getDeviceIDs()
sn.deviceIDs

['1039212', '', '', '', '', '', '', '']

In [ ]:
XY_scan(xdrive,ydrive,0,1,10,0,1,10,1,'Trial')

In [12]:
sn.setPTUFilePath("./PTU.ptu")
sn.histogram.setRefChannel(0)
sn.histogram.setBinWidth(10000)
sn.histogram.measure(10000,savePTU=True)
data, bins = sn.histogram.getData()